# عوامل هوش مصنوعی Azure با پشتیبانی از پروتکل زمینه مدل (MCP) - پایتون

این دفترچه نشان می‌دهد که چگونه می‌توان از عوامل هوش مصنوعی Azure با ابزارهای پروتکل زمینه مدل (MCP) در پایتون استفاده کرد. این مثال نحوه ایجاد یک عامل هوشمند را نشان می‌دهد که می‌تواند از سرورهای خارجی MCP (مانند Microsoft Learn) برای قابلیت‌های پیشرفته با استفاده از احراز هویت بدون کلید بهره‌مند شود.


## نصب بسته‌های مورد نیاز پایتون

ابتدا باید بسته‌های ضروری پایتون را نصب کنیم:
- **azure-ai-projects**: هسته SDK پروژه‌های Azure AI
- **azure-ai-agents**: SDK عوامل Azure AI برای ایجاد و مدیریت عوامل
- **azure-identity**: احراز هویت بدون کلید با استفاده از DefaultAzureCredential
- **mcp**: پیاده‌سازی پروتکل Model Context برای پایتون


## مزایای احراز هویت بدون کلید

این دفترچه **احراز هویت بدون کلید** را نشان می‌دهد که چندین مزیت ارائه می‌دهد:
- ✅ **بدون نیاز به مدیریت کلیدهای API** - از احراز هویت مبتنی بر هویت Azure استفاده می‌کند
- ✅ **امنیت بیشتر** - هیچ راز یا اطلاعات حساسی در کد یا فایل‌های پیکربندی ذخیره نمی‌شود
- ✅ **چرخش خودکار اعتبارنامه‌ها** - Azure مدیریت چرخه عمر اعتبارنامه‌ها را بر عهده دارد
- ✅ **کنترل دسترسی مبتنی بر نقش** - از RBAC Azure برای تنظیم دقیق مجوزها استفاده می‌کند
- ✅ **پشتیبانی از چند محیط** - به‌طور یکپارچه در محیط‌های توسعه و تولید کار می‌کند

`DefaultAzureCredential` به‌طور خودکار بهترین منبع اعتبارنامه موجود را انتخاب می‌کند:
1. **Managed Identity** (زمان اجرا در Azure)
2. اعتبارنامه‌های **Azure CLI** (در طول توسعه محلی)
3. اعتبارنامه‌های **Visual Studio**
4. **متغیرهای محیطی** (در صورت پیکربندی)
5. احراز هویت **مرورگر تعاملی** (به‌عنوان گزینه جایگزین)


## تنظیم احراز هویت بدون کلید

**پیش‌نیازهای احراز هویت بدون کلید:**

### برای توسعه محلی:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### برای محیط‌های Azure:
- فعال کردن **System-assigned Managed Identity** روی منبع Azure خود
- اختصاص **نقش‌های RBAC** مناسب به هویت مدیریت‌شده:
  - `Cognitive Services OpenAI User` برای دسترسی به Azure OpenAI
  - `AI Developer` برای دسترسی به پروژه‌های Azure AI

### متغیرهای محیطی (اختیاری):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**نیازی به کلیدهای API یا رشته‌های اتصال نیست!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## وارد کردن کتابخانه‌های مورد نیاز

ماژول‌های پایتون مورد نیاز را وارد کنید:
- **os, time**: کتابخانه‌های استاندارد پایتون برای متغیرهای محیطی و تأخیرها
- **AIProjectClient**: کلاینت اصلی برای پروژه‌های هوش مصنوعی Azure
- **DefaultAzureCredential**: احراز هویت بدون کلید برای خدمات Azure
- **کلاس‌های مرتبط با MCP**: برای ایجاد و مدیریت ابزارهای MCP و رسیدگی به تأییدیه‌ها


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## تنظیمات سرور MCP را پیکربندی کنید

پیکربندی سرور MCP را با استفاده از متغیرهای محیطی و مقادیر پیش‌فرض انجام دهید:
- **MCP_SERVER_URL**: آدرس URL سرور MCP (به‌صورت پیش‌فرض API Microsoft Learn)
- **MCP_SERVER_LABEL**: یک برچسب برای شناسایی سرور MCP (به‌صورت پیش‌فرض "mslearn")

این روش امکان پیکربندی انعطاف‌پذیر در محیط‌های مختلف را فراهم می‌کند.


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## ایجاد کلاینت پروژه Azure AI (احراز هویت بدون کلید)

کلاینت پروژه Azure AI را با استفاده از **احراز هویت بدون کلید** مقداردهی اولیه کنید:
- **endpoint**: آدرس URL نقطه پایانی پروژه Azure AI Foundry
- **credential**: از `DefaultAzureCredential()` برای احراز هویت امن و بدون کلید استفاده می‌کند
- **بدون نیاز به کلیدهای API**: به‌طور خودکار بهترین اعتبار موجود را کشف و استفاده می‌کند

**جریان احراز هویت:**
1. بررسی هویت مدیریت‌شده (در محیط‌های Azure)
2. استفاده از اعتبارنامه‌های Azure CLI (برای توسعه محلی)
3. استفاده از سایر منابع اعتبارنامه موجود در صورت نیاز

این روش نیاز به مدیریت کلیدهای API یا رشته‌های اتصال در کد شما را از بین می‌برد.


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## ایجاد تعریف ابزار MCP

یک ابزار MCP ایجاد کنید که به سرور MCP مایکروسافت لرن متصل شود:
- **server_label**: شناسه‌ای برای سرور MCP
- **server_url**: نقطه پایانی URL سرور MCP
- **allowed_tools**: لیست اختیاری برای محدود کردن ابزارهایی که می‌توانند استفاده شوند (لیست خالی به همه ابزارها اجازه می‌دهد)

این ابزار به عامل اجازه می‌دهد به مستندات و منابع مایکروسافت لرن دسترسی پیدا کند.


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## ایجاد عامل و اجرای مکالمه (جریان کاری بدون کلید)

این بخش جامع، **جریان کاری عامل بدون کلید** را به طور کامل توضیح می‌دهد:

1. **ایجاد عامل هوش مصنوعی**: تنظیم یک عامل با مدل GPT-4.1 nano و ابزارهای MCP  
2. **ایجاد رشته مکالمه**: برقراری یک رشته مکالمه برای ارتباط  
3. **ارسال پیام**: از عامل درباره تفاوت‌های Azure OpenAI و OpenAI سؤال کنید  
4. **مدیریت تأیید ابزارها**: تأیید خودکار فراخوانی ابزارهای MCP در صورت نیاز  
5. **نظارت بر اجرا**: پیگیری پیشرفت عامل و مدیریت اقدامات مورد نیاز  
6. **نمایش نتایج**: نمایش جزئیات مکالمه و استفاده از ابزارها  

**ویژگی‌های بدون کلید:**
- ✅ **بدون اسرار سخت‌کد شده** - تمام احراز هویت از طریق هویت Azure مدیریت می‌شود  
- ✅ **امنیت پیش‌فرض** - از کنترل دسترسی مبتنی بر نقش استفاده می‌کند  
- ✅ **استقرار ساده‌شده** - نیازی به مدیریت اعتبارنامه نیست  
- ✅ **مناسب برای ممیزی** - تمام دسترسی‌ها از طریق هویت Azure ردیابی می‌شود  

عامل از ابزارهای MCP برای دسترسی به منابع Microsoft Learn با امنیت کامل و بدون نیاز به مدیریت کلید API استفاده خواهد کرد.


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما تلاش می‌کنیم دقت را حفظ کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادرستی‌ها باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، توصیه می‌شود از ترجمه حرفه‌ای انسانی استفاده کنید. ما مسئولیتی در قبال سوء تفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
